## Keras

Keras это высокоуровневая обертка над tensorflow. Но в текущих реализациях библиотеке tf они живут очень рядом. 
Когда мы собираем свои нейронки мы берем уже готовые слои из keras и добавляем что-то свое, если нам требуется.
Но keras можно использовать без явного использования TF пытаясь свести задачу к fit-predict.


### Sequential
Самое простое, что мы можем сделать это собирать слои последовательно друг за другом - займеся же этим!

In [1]:
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Sequential, layers as L # подгружаем нужные модули. 
import tensorflow.keras as keras

In [2]:
# в keras лежит несколько наборов данных. Для примера возьмем fashion_mnist - как mnist, но про предметы одежды :)
fashion_mnist = tf.keras.datasets.fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=10**4, random_state=42)

X_train = X_train/ 255.
X_val = X_val/ 255.
X_test = X_test/ 255.

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [3]:
# для того, чтобы учить через cross_entropy нам нужно сделать OHE таргетам. И эта функция есть в keras!
from tensorflow.keras.utils import to_categorical

y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)
y_val_ohe = to_categorical(y_val)


In [4]:
## Первая простая нейронка

tf.random.set_seed(42) # фиксируем random_seed

model = Sequential(name = 'first_try')
model.add(L.Input(shape = (28,28))) # входной нейрон с данными. Его обычно можно опускать, сразу передавая 
# в нейрон размерность. Но Dense ячейки не умеют работать с картинками, поэтому оставляем Input
model.add(L.Flatten()) # разворачиваем картинку в вектор
model.add(L.Dense(100,  kernel_initializer='random_normal',name='First')) # можно именовать и потом брать слои по именам
model.add(L.ReLU()) # добавляем активацию
model.add(L.Dense(10, kernel_initializer = 'random_normal',name='Output'))
model.add(L.Softmax())
opt = keras.optimizers.Adam(learning_rate=1e-4) # так же нам нужно указать оптимайзер
model.compile(optimizer=opt,loss='categorical_crossentropy',
             metrics=["categorical_accuracy"]) # и собрать нашу модель, указав метрики,loss и оптимизатор


history1 = model.fit(X_train,y_train_ohe,batch_size=500,epochs=2,validation_data = (X_val,y_val_ohe))
# и процесс обучения. Задаем количество эпох, размер батча и валидационную часть наших данных


Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 1s 14us/sample - loss: 1.8986 - categorical_accuracy: 0.4763 - val_loss: 1.4738 - val_categorical_accuracy: 0.6307
Epoch 2/2
50000/50000 [==============================] - 0s 8us/sample - loss: 1.2020 - categorical_accuracy: 0.6655 - val_loss: 1.0251 - val_categorical_accuracy: 0.6802


In [5]:
#history1.params

In [6]:
#history1.history

In [7]:
# Эту же модель можно записать чуть в меньшее количество строчек кода
model = Sequential(name = 'first_try')
model.add(L.Input(shape = (28,28))) 
model.add(L.Flatten()) 
model.add(L.Dense(100,  kernel_initializer='random_normal',name='First',activation='relu')) # можно именовать и потом брать слои по именам
model.add(L.Dense(10, kernel_initializer = 'random_normal',name='Output',activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=1e-4) 
model.compile(optimizer=opt,loss='categorical_crossentropy',
             metrics=["categorical_accuracy"]) 

history1 = model.fit(X_train,y_train_ohe,batch_size=500,epochs=2,validation_data = (X_val,y_val_ohe))


Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 1s 15us/sample - loss: 1.8141 - categorical_accuracy: 0.4979 - val_loss: 1.3973 - val_categorical_accuracy: 0.6449
Epoch 2/2
50000/50000 [==============================] - 0s 8us/sample - loss: 1.1440 - categorical_accuracy: 0.6720 - val_loss: 0.9782 - val_categorical_accuracy: 0.6839


In [8]:
# ?L.Dense

Из приятного - все в таком подходе можно кастомизировать под себя!
Но пока продолжим рассмотрение о следующем подходе сборки моделей.
Класс Sequential не дает нам вообще никакой гибкости, позволяя набирать слои только последовательно.
Что же у нас есть новый герой  - Model.
Он позволяет собирать сетки практически любой архитектуры

In [9]:
from tensorflow.keras import Model  # подгружаем нужные модули. 

In [16]:
init = 'uniform'
act = 'relu'

input_tensor = L.Input(shape=(28, 28)) # задаем вход
x = L.Flatten()(input_tensor)# применение нейрона к входу
x = L.Dense(100, kernel_initializer=init, activation=act)(x) # повторяем всю логику сколько нам надо
x = L.Dense(100, kernel_initializer=init, activation=act)(x)
output_tensor = L.Dense(10, kernel_initializer=init, activation='softmax')(x)

model = keras.Model(input_tensor, output_tensor) # Keras под копотом сам собирает граф.
# Если он может получить из входа выхода то вы великолепны.

model.compile(optimizer=opt,loss='categorical_crossentropy',
             metrics=["categorical_accuracy"]) 

In [17]:
history = model.fit(X_train,y_train_ohe,batch_size=500,epochs=2,validation_data = (X_val,y_val_ohe))

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 1s 15us/sample - loss: 1.7141 - categorical_accuracy: 0.4943 - val_loss: 1.1252 - val_categorical_accuracy: 0.6275
Epoch 2/2
50000/50000 [==============================] - 0s 8us/sample - loss: 0.9337 - categorical_accuracy: 0.6636 - val_loss: 0.8333 - val_categorical_accuracy: 0.6795


Такой подход позволяет делать практически любой гибкости нейронки. Как пример - двухголовая!

In [24]:
input_1 = L.Input(shape=(28, 28))
input_2 = L.Input(shape=(28, 28))

x1 = L.Flatten()(input_1)
x1 = L.Dense(100, kernel_initializer=init, activation=act)(x1)
x1 = L.Dense(100, kernel_initializer=init, activation=act)(x1)

x2 = L.Flatten()(input_2)
x2 = L.Dense(100, kernel_initializer=init, activation=act)(x2)
x2 = L.Dense(100, kernel_initializer=init, activation=act)(x2)

x = L.concatenate([x1, x2]) # Волшебное слово, которое позволяет нам соеденять несколько потоков наших данных
output = L.Dense(10, kernel_initializer=init, activation='softmax')(x)

model = keras.Model([input_1, input_2], output)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
flatten_10 (Flatten)            (None, 784)          0           input_11[0][0]                   
__________________________________________________________________________________________________
flatten_11 (Flatten)            (None, 784)          0           input_12[0][0]                   
____________________________________________________________________________________________

In [25]:
model.compile(optimizer=opt,loss='categorical_crossentropy',
             metrics=["categorical_accuracy"]) 

In [27]:
history = model.fit([X_train,X_train],y_train_ohe,batch_size=500,epochs=2,validation_data = ([X_val,X_val],y_val_ohe))

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 1s 22us/sample - loss: 1.3714 - categorical_accuracy: 0.6164 - val_loss: 0.8260 - val_categorical_accuracy: 0.6711
Epoch 2/2
50000/50000 [==============================] - 1s 14us/sample - loss: 0.7213 - categorical_accuracy: 0.7305 - val_loss: 0.6702 - val_categorical_accuracy: 0.7558


In [12]:
# нужно для винды, если не видит путь до graphviz
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
from tensorflow.keras.utils import plot_model


In [13]:
plot_model(model) # можно нарисовать модельку

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


## 4.2 Несколько выходов и функций потерь

In [20]:
init = 'uniform'
act = 'relu'

input_tensor = L.Input(shape=(28, 28))

x = L.Flatten()(input_tensor)
x1 = L.Dense(100, kernel_initializer=init, activation=act)(x)
x2 = L.Dense(100, kernel_initializer=init, activation=act)(x)
x3 = L.Dense(100, kernel_initializer=init, activation=act)(x)

output_1 = L.Dense(1, kernel_initializer=init, activation='sigmoid',name='gender')(x1)
output_2 = L.Dense(10, kernel_initializer=init, activation='softmax',name='income')(x2)
output_3 = L.Dense(1, kernel_initializer=init,name='age')(x3)

model = keras.Model(input_tensor, [output_1, output_2, output_3])

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
flatten_9 (Flatten)             (None, 784)          0           input_10[0][0]                   
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 100)          78500       flatten_9[0][0]                  
__________________________________________________________________________________________________
dense_27 (Dense)                (None, 100)          78500       flatten_9[0][0]                  
____________________________________________________________________________________________

In [21]:
# чтобы модель не переобучилась под самую большую функцию потерь
# их можно взвесить
model.compile(optimizer='adam', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
                                    loss_weights=[0.25, 1., 10.])


# если дали выходам имена, можно вот так: 

model.compile(optimizer='adam',

              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
                    
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

Помните статью про то, как люди рисовали функции потерь? [Теперь появилась галерея!](https://losslandscape.com/gallery/) Есть подход как skip-connection, он сильно меняет нашу функцию.

![](https://i.stack.imgur.com/UDvbg.png)

Такую модель нельзя собрать через `Sequence`-стиль, но можно через функциональный стиль. Давайте попробуем сделать это. Заодно посмотрим насколько сильно в нашей ситуации будет меняться траектория обучения. (Сравним обычный 6-ти слойный персептрон и с прокидыванием инфы, например со 2 слоя на 5ый). Ну и также сразу поэксперементируем с функциями активаций и batchnorm.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ  создайте сеть по инструкциям ниже
tf.random.set_seed(42)
## Соберите обычный 6ти слойный перспептрон



In [ ]:
## Соберите обычный 6ти слойный перспептрон с функциями активации сигмоид


In [ ]:
## Соберите с батчнормом и релу


In [ ]:
## Соберите ну и наконец прокидывание данных


In [28]:
# Функция для удобной отрисовки всего

def plot_history(histories, key='loss', start=0):
    plt.figure(figsize=(16,10))

    for name, history in histories:
        val = plt.plot(history.epoch[start:], history.history['val_'+key][start:],
                       #'--', 
                       label=name.title()+' Val')
            #plt.plot(history.epoch[start:], history.history[key][start:], color=val[0].get_color(),
            #     label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([start, max(history.epoch)])


In [ ]:
plot_history([('simple',    history_simple), 
              ('simple_sigmoid', history_simple_sigmoid),
              ('simple_BN_and_init', history_simple_BN_and_init),
              ('complex!!!',   history_complex)
             ],
             start=0)

#### Более интересная часть - кастомизация
В keras уже есть несколько удобных callback.

In [30]:
from tensorflow.keras.callbacks import EarlyStopping,LearningRateScheduler,ReduceLROnPlateau,ModelCheckpoint

In [31]:
EarlyStopping # останавливает обучение если наша метрика не меняет n эпох
LearningRateScheduler # меняет наш learning_rate по расписанию
ReduceLROnPlateau # понижает на LR если не происходит улучшения
ModelCheckpoint # сохраняет нашу лучшую модель

tensorflow.python.keras.callbacks.ModelCheckpoint

In [42]:
early_stop = EarlyStopping(patience=3)
reduce_on_plateau = ReduceLROnPlateau(patience=3)
# filepath="checkpoint_path/weights-improvement-{epoch:02d}-{val_categorical_accuracy:.2f}.hdf5"
filepath="checkpoint_path/weights-improvement.hdf5"
model_checkpoing = ModelCheckpoint(filepath,
                                   save_best_only=True,
                                  save_weights_only=True)

In [57]:
def create_simple_model():
    model = Sequential(name = 'simple_model')
    model.add(L.Input(shape = (28,28))) 
    model.add(L.Flatten()) 
    model.add(L.Dense(100,  kernel_initializer='random_normal',name='First',activation='relu'))
    model.add(L.Dense(100,  kernel_initializer='random_normal',name='Second',activation='relu'))
    model.add(L.Dense(10, kernel_initializer = 'random_normal',name='Output',activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=1e-4) 
    model.compile(optimizer=opt,loss='categorical_crossentropy',
                 metrics=["categorical_accuracy"]) 
    return model


In [59]:

model = create_simple_model()

history = model.fit(X_train,y_train_ohe,batch_size=500,epochs=20,
                     validation_data = (X_val,y_val_ohe),
                    callbacks = [early_stop,reduce_on_plateau,model_checkpoing],verbose=0)


In [44]:
model.load_weights('checkpoint_path/weights-improvement.hdf5')

In [60]:
filepath="checkpoint_path/full_model_improvement.hdf5"
model_checkpoing = ModelCheckpoint(filepath,
                                   save_best_only=True,
                                  save_weights_only=False)
simple_model = create_simple_model()

history = simple_model.fit(X_train,y_train_ohe,batch_size=500,epochs=20,
                     validation_data = (X_val,y_val_ohe),
                    callbacks = [early_stop,reduce_on_plateau,model_checkpoing],verbose=0)


In [61]:
simple_model = keras.models.load_model(filepath)

In [62]:
simple_model.evaluate(x=X_val,y=y_val_ohe)

10000/10000 [==============================] - 0s 26us/sample - loss: 0.4358 - categorical_accuracy: 0.8454


[0.43579934401512144, 0.8454]

In [63]:
from tensorflow.keras import callbacks
 
class My_Callback(callbacks.Callback):     # Класс My_Callback унаследовал свойства класса Callback
    def on_train_begin(self, logs={}):           # Функция, которая выполняется в начале обучения 
        return
 
    def on_train_end(self, logs={}):             # Функция, которая выполняется в конце обучения 
        return
 
    def on_epoch_begin(self, logs={}):           # В начале каждой эпохи 
        return
 
    def on_epoch_end(self, epoch, logs={}): 
        # В конце каждой эпохи
        return
 
    def on_batch_begin(self, batch, logs={}):    # В начале батча
        return
 
    def on_batch_end(self, batch, logs={}):      # В конце батча 
        return

In [64]:
from tensorflow.keras import callbacks

class Printlogs(callbacks.Callback):

    def on_epoch_end(self, epoch, logs):
        if epoch==10:
            print(logs)
        

In [67]:
simple_model = create_simple_model()
our_callback = Printlogs()

In [68]:
history = simple_model.fit(X_train,y_train_ohe,batch_size=500,epochs=20,
                     validation_data = (X_val,y_val_ohe),
                    callbacks = [our_callback],
                           verbose=0)

{'loss': 0.47610742747783663, 'categorical_accuracy': 0.83594, 'val_loss': 0.4837979659438133, 'val_categorical_accuracy': 0.8317}


In [84]:
import numpy as np
# Напишем изменение скорости обучения
INIT_LR=0.1
# Стратегия для понижения скорости
def lr_scheduler(epoch):
    drop = 0.5
    epochs_drop = 1.0
    lrate = INIT_LR * np.math.pow(drop, np.math.floor((epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(lr_scheduler)
# класс чтобы отслеживать бесчинства
class Print_lr(callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print(float(tf.keras.backend.get_value(self.model.optimizer.lr)))
        # чтобы установить свой LR надо указать
        # LR_OUR = ....
        # tf.keras.backend.set_value(self.model.optimizer.lr, LR_OUR)


simple_model = create_simple_model()
history = simple_model.fit(X_train,y_train_ohe,batch_size=500,epochs=20,
                     validation_data = (X_val,y_val_ohe),
                    callbacks = [lrate,Print_lr()],
                           verbose=0)


0.10000000149011612
0.05000000074505806
0.02500000037252903
0.012500000186264515
0.0062500000931322575
0.0031250000465661287
0.0015625000232830644
0.0007812500116415322
0.0003906250058207661
0.00019531250291038305
9.765625145519152e-05
4.882812572759576e-05
2.441406286379788e-05
1.220703143189894e-05
6.10351571594947e-06
3.051757857974735e-06
1.5258789289873675e-06
7.629394644936838e-07
3.814697322468419e-07
1.9073486612342094e-07


In [ ]:
# Давайте заставим модель посчитать метрики каждую эпоху, как пример
# Считать будем на X_val, y_val

### Кастомные loss и метрики
Что уже есть
https://keras.io/api/metrics/

https://keras.io/api/losses/

In [ ]:
celsius    = np.array([-40, -10,  0,  8, 15, 22,  38],  dtype=float)
fahrenheit = np.array([-40,  14, 32, 46, 59, 72, 100],  dtype=float)

In [95]:


model = Sequential()
model.add(L.Dense(1))

opt = tf.keras.optimizers.Adam( )

model.compile(loss='mse', optimizer=opt)
model.fit(celsius, fahrenheit,  epochs=3, verbose=1)



Wall time: 427 ms


In [91]:
def custom_loss_function(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

In [98]:
%%time
model = Sequential()
model.add(L.Dense(1))

opt = tf.keras.optimizers.Adam( )

model.compile(loss=custom_loss_function, optimizer=opt)
model.fit(celsius, fahrenheit,  epochs=3,verbose=1)



Train on 7 samples
Epoch 1/3
7/7 [==============================] - 0s 26ms/sample - loss: 1291.6083
Epoch 2/3
7/7 [==============================] - 0s 143us/sample - loss: 1290.7059
Epoch 3/3
7/7 [==============================] - 0s 142us/sample - loss: 1289.8048
Wall time: 233 ms


In [97]:
## тоже самое можно делать и с метриками. Также можно следить сразу за несколькими метриками, что бывает полезно.
## Если хотим добавить совсем сложную логику то мы это будем делать через callback

В этой тетрадке немного поработаем с градусами по цельсию и фаренгейту! Снова попробуем восстановить формулу 

$$ f = c \times 1.8 + 32 $$

In [155]:
## возьмем срезы
tf.keras.backend.set_floatx('float64')
model = Sequential()
model.add(L.Dense(1,name='our_neural'))

opt = tf.keras.optimizers.Adam(0.1 )

model.compile(loss='mse', optimizer=opt)
model.fit(celsius, fahrenheit,  epochs=600, verbose=0)



In [156]:
## элементарная задача, но из-за того, что данные не скалированы сходились вечность
our_layer = model.get_layer(name='our_neural')
our_layer.variables

[<tf.Variable 'sequential_25/our_neural/kernel:0' shape=(1, 1) dtype=float64, numpy=array([[1.80844236]])>,
 <tf.Variable 'sequential_25/our_neural/bias:0' shape=(1,) dtype=float64, numpy=array([30.71169356])>]

In [157]:
our_layer(np.array(celsius).reshape((7,1))) ## Берем прогнозы от слоя

<tf.Tensor: shape=(7, 1), dtype=float64, numpy=
array([[-41.62600081],
       [ 12.62726997],
       [ 30.71169356],
       [ 45.17923244],
       [ 57.83832895],
       [ 70.49742546],
       [ 99.43250321]])>

 Есть понимание, как взять определить в callback на какой эпохе мы получили правильное значение весов? Проверить как ведет себя наша нейронка? (с учетом того, что мы точно знаем формулу)
 Ну и заодно быстро проверить гипотезу - а поможет ли нам batchnorm в данной ситуации?

### Пишем класс нейронки с TF и keras вместе

In [193]:
# транспонировали выборку
x_train = celsius[:,None]
y_train = fahrenheit[:,None]

In [194]:
tf.keras.backend.set_floatx('float64')
class Super_puper_neural_net(Model):
    
    def __init__(self, n_hidden_neurons):
        super(Super_puper_neural_net, self).__init__()
        self.fc1 = L.Dense(n_hidden_neurons, kernel_initializer='glorot_uniform',
                           activation='relu', trainable=True)
        self.fc2 = L.Dense(n_hidden_neurons, kernel_initializer='glorot_uniform',
                           trainable=True)

    def encode(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [197]:
model_super = Super_puper_neural_net(1)
model_super.encode(x_train)

<tf.Tensor: shape=(7, 1), dtype=float64, numpy=
array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 3.38618427],
       [ 6.34909551],
       [ 9.31200675],
       [16.0843753 ]])>

In [198]:
# Ошибка для модели
def mean_square(y_pred, y_true):
    return tf.reduce_mean((y_pred-y_true)**2)

# оптимизатор 
optimizer = tf.optimizers.SGD(learning_rate=0.001)

# процесс оптимизации
def model_train(X, Y):

    # находим loss и пробрасываем градиент
    with tf.GradientTape() as g:
        pred = model_super.encode(X)
        loss = mean_square(pred, Y)

    # Вычисляем градиенты
    gradients = g.gradient(loss, model_super.variables)
    
    # Обновляем веса a и b в ходе одной итерации спуска 
    optimizer.apply_gradients(zip(gradients, model_super.variables))

In [200]:
#Обучение
epochs = 1000 # число эпох 

for i in range(epochs):
    
    # Делаем щаг градиентного спуска 
    model_train(x_train, y_train)
    
    # Каждую сотую итерацию следим за тем, что произошло
    if i%100 == 0:
        y_pred = model_super.encode(x_train)
        loss_val = mean_square(y_pred, y_train)
        print("step: %i, loss: %f" % (i, loss_val))

step: 0, loss: 2452.378595
step: 100, loss: 1908.733339
step: 200, loss: 1512.925791
step: 300, loss: 1224.864457
step: 400, loss: 1015.159668
step: 500, loss: 862.328840
step: 600, loss: 750.747670
step: 700, loss: 669.096943
step: 800, loss: 654.442778
step: 900, loss: 630.576029


#  Свой слой на Tensorflow для Keras

Новые слои можно писать на основе керасовского класса `Layer`. Если прописать `help(tf.keras.layers.Layer)`, можно почитать про него. Если в кратце, нужно реализовать три части: 

* Конструктор, в нём мы описываем гиперпараметры 
* Метод `build`, в которм мы описываем все переменные 
* Метод `call`, который делает forward pass

In [175]:
class MyLinear(L.Layer):
    
    # Задаём консруктор 
    def __init__(self, units=32):
        super(MyLinear, self).__init__()  # чтобы коректно унаследовались методы
        self.units = units                # число нейронов

    def build(self, input_shape):
        # add_weight внутри build то же самое что и Variable, но совместимо с Keras
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal', 
                                 trainable=True)
        
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal', 
                                 trainable=True)

    # Применение 
    def call(self, inputs):
        # сразу делаем и линейное преобразование и ReLU (а почему бы и нет)
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b) 

In [179]:
model_custom = Sequential(name = 'simple_model')
model_custom.add(L.Input(shape = (28,28))) 
model_custom.add(L.Flatten()) 
model_custom.add(L.Dense(100,  kernel_initializer='random_normal',name='First',activation='relu'))
model_custom.add(MyLinear()) ### Самый красивый слой
model_custom.add(L.Dense(10, kernel_initializer = 'random_normal',name='Output',activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=1e-4) 
model_custom.compile(optimizer=opt,loss='categorical_crossentropy',
             metrics=["categorical_accuracy"]) 

In [182]:
history = model_custom.fit(X_train,y_train_ohe,batch_size=500,epochs=20,
                     validation_data = (X_val,y_val_ohe),
                    callbacks = [early_stop,reduce_on_plateau],verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 1s 12us/sample - loss: 0.6633 - categorical_accuracy: 0.7723 - val_loss: 0.6594 - val_categorical_accuracy: 0.7683
Epoch 2/20
50000/50000 [==============================] - 1s 12us/sample - loss: 0.6257 - categorical_accuracy: 0.7865 - val_loss: 0.6268 - val_categorical_accuracy: 0.7819
Epoch 3/20
50000/50000 [==============================] - 1s 11us/sample - loss: 0.5967 - categorical_accuracy: 0.7961 - val_loss: 0.6017 - val_categorical_accuracy: 0.7942
Epoch 4/20
50000/50000 [==============================] - 1s 11us/sample - loss: 0.5729 - categorical_accuracy: 0.8044 - val_loss: 0.5803 - val_categorical_accuracy: 0.8012
Epoch 5/20
50000/50000 [==============================] - 1s 12us/sample - loss: 0.5536 - categorical_accuracy: 0.8101 - val_loss: 0.5622 - val_categorical_accuracy: 0.8095
Epoch 6/20
50000/50000 [==============================] - 1s 12us/sample - loss: 0.53

Ну и нам остался пример, как взять срез модели. Посмотреть прогнозы в середине

In [159]:
model = create_simple_model()

In [160]:
history = model.fit(X_train,y_train_ohe,batch_size=500,epochs=20,
                     validation_data = (X_val,y_val_ohe),
                    callbacks = [early_stop,reduce_on_plateau],verbose=0)

In [171]:
# Извлекаем выходы верхних 2х слоев
layer_outputs = [layer.output for layer in model.layers[1:3]]
# создаем модель, которая вернет эти выходы с учетом заданнаго входа
activation_model = Model(inputs=model.input, outputs=layer_outputs)

In [172]:
prediction = activation_model.predict(X_val)

In [173]:
prediction

[array([[1.38649075, 3.54336526, 3.78493653, ..., 4.28923992, 2.59533527,
         2.33044051],
        [0.84572429, 4.38319305, 2.8557391 , ..., 2.79402664, 1.15542673,
         1.56231554],
        [1.88866113, 0.49722533, 0.48871935, ..., 3.56315018, 4.26814976,
         1.25973122],
        ...,
        [1.64454343, 0.61663136, 0.        , ..., 0.94446987, 1.6462323 ,
         4.40703848],
        [1.66231301, 0.92270261, 1.71392342, ..., 2.53571798, 2.92817679,
         1.16723328],
        [1.46921835, 0.65876474, 1.72670742, ..., 3.23719186, 3.7936972 ,
         1.68555778]]),
 array([[5.5452259 , 1.21933981, 3.04301914, ..., 0.37099667, 3.9048873 ,
         4.75640925],
        [6.6589373 , 0.96440545, 4.043728  , ..., 0.        , 6.31727883,
         6.48483602],
        [5.0319833 , 0.12222322, 0.        , ..., 2.18646237, 0.84729714,
         3.49273428],
        ...,
        [0.        , 5.69175408, 0.25687616, ..., 0.31629389, 1.55848377,
         0.        ],
        [3.5

In [174]:
layer_outputs

[<tf.Tensor 'First_31/Identity:0' shape=(None, 100) dtype=float64>,
 <tf.Tensor 'Second_29/Identity:0' shape=(None, 100) dtype=float64>]

 Что сегодня не вошло - как переопределить градиенты для своих слоев (на уровне keras очень геморойно, если уже занимаетесь этим то вряд ли пишете на верхнеуровневом фраемворке)
Как работать с уже готовыми и обучеными моделями, дофичивать нейронки по кусочкам. Но это уже в следующих сериях :)

